Importing the Pandas and grabbing the table from the wiki page

In [ ]:
import pandas as pd

In [6]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url, header=0,keep_default_na=False)
tdf = table[0]
tdf.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Fixing up the column names

In [7]:
tdf.columns = ['PostalCode','Borough','Neighborhood'] 
tdf.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Getting rid of unassigned boroughs

In [8]:
tdf1 = tdf.query('Borough != "Not assigned"').reset_index(drop=True) 
tdf1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


If there is more than one neighborhood in the same postalcode, combine the neighborhood separated by commas

In [9]:
tdf2=tdf1.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
tdf2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
tdf2.loc[tdf2['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = tdf2['Borough']
tdf2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


To see number of rows in the data frame

In [15]:
tdf2.shape[0]

103